# Module imports
Проверь, все ли используются. Ненужные - удали.

In [52]:
from catboost.datasets import titanic # Учебный датасет
from catboost import CatBoostClassifier, Pool, metrics, cv
from catboost import MetricVisualizer
import numpy as np
from numpy.random import default_rng
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, log_loss, precision_score, recall_score, roc_auc_score #, roc_curve, auc # Не получилось запустить
import hyperopt

# Загружаем данные
Тут используем учебный датасет.

Создаём два датасета - для обучения модели и для проверки.

In [53]:
train_df, test_df = titanic()

Изучаем наш датасет.\
\
Что находится в колонках:\
PassengerId - порядковый номер пассажира. Просто инкремент. Не является данными.\
Survived - выжил или нет. Целевая метрика. 0 = Нет, 1 = Да.\
Pclass - Ticket class, класс обслуживания. 1 = 1st, 2 = 2nd, 3 = 3rd.\
Sex - пол.\
Age - возраст.\
SibSp - siblings / spouses, кол-во сопровождающих братьев / супругов.\
Parch - parents / children, кол-во сопровождающих детей / родителей.\
Ticket - номер билета.\
Fare - стоимость билета.\
Cabin - номер каюты.\
Embarked - Port of Embarkation, порт посадки. C = Cherbourg, Q = Queenstown, S = Southampton.\

Удалим колонку 'PassengerId'.

In [54]:
# train_df.drop('PassengerId', axis=1, inplace=True)
# test_df.drop('PassengerId', axis=1, inplace=True)

In [55]:
train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [56]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [57]:
print('Survived: ' + str(set(train_df['Survived'])))
print('Pclass: ' + str(set(train_df['Pclass'])))
print('SibSp: ' + str(set(train_df['SibSp'])))
print('Parch: ' + str(set(train_df['Parch'])))
print('Fare: ' + str(set(train_df['Fare'])))
print('Embarked: ' + str(set(train_df['Embarked'])))


Survived: {0, 1}
Pclass: {1, 2, 3}
SibSp: {0, 1, 2, 3, 4, 5, 8}
Parch: {0, 1, 2, 3, 4, 5, 6}
Fare: {0.0, 512.3292, 4.0125, 5.0, 6.975, 7.925, 8.05, 7.25, 8.4583, 11.1333, 7.8542, 13.0, 7.225, 8.0292, 16.7, 16.0, 18.0, 11.2417, 15.5, 21.075, 21.0, 21.6792, 17.8, 20.575, 26.55, 26.0, 27.7208, 29.125, 30.0708, 31.275, 31.3875, 27.75, 27.9, 35.5, 7.75, 29.0, 34.375, 39.6875, 34.6542, 41.5792, 7.1417, 36.75, 7.7958, 7.3125, 46.9, 9.0, 9.5, 47.1, 50.0, 51.8625, 10.5, 53.1, 52.0, 55.0, 56.4958, 11.5, 52.5542, 56.9292, 57.9792, 61.9792, 61.175, 63.3583, 61.3792, 12.875, 66.6, 13.5, 12.0, 69.55, 69.3, 71.2833, 14.4542, 73.5, 14.5, 14.0, 76.7292, 77.2875, 15.0458, 79.2, 80.0, 16.1, 82.1708, 83.475, 15.75, 76.2917, 86.5, 79.65, 83.1583, 81.8583, 90.0, 18.7875, 91.0792, 18.75, 89.1042, 93.5, 19.5, 19.2583, 7.2292, 20.525, 20.2125, 20.25, 7.7292, 106.425, 13.8583, 108.9, 110.8833, 22.3583, 22.025, 113.275, 22.525, 23.0, 23.25, 120.0, 24.15, 24.0, 25.5875, 25.9292, 26.25, 26.2875, 133.65, 134.5, 135

# Подготовка данных
Проверяем пустые значения. Смотрим в каких колонках и как много.

In [58]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Age         177
Cabin       687
Embarked      2
dtype: int64

Если много пустых значений, то это запутает модель. Нужно выбрать решение:
- Удалить строки
- Проанализировать и заполнить на сонове косвенных данных
- Заполнить таким значением, чтобы это не повлияло

Поле 'Embarked'

In [59]:
train_df[train_df['Embarked'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [60]:
embarked_values_count = train_df['Embarked'].value_counts().reset_index()
embarked_values_count.columns = ['Embark', 'Count']

print(embarked_values_count)

  Embark  Count
0      S    644
1      C    168
2      Q     77


Пропущено всего 2 значения. Какое бы значение мы не применили, это не сильно повлияет на результат. Заменим пустые на самое распространенное значение.

In [61]:
train_df['Embarked'] = train_df['Embarked'].fillna('S')

In [62]:
train_df[train_df['Age'].isna()].sort_values(by='Ticket')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
475,476,0,1,"Clifford, Mr. George Quincy",male,NaN,0,0,110465,52.00,A14,S
507,508,1,1,"Bradley, Mr. George (""George Arthur Brayton"")",male,NaN,0,0,111427,26.55,NaN,S
633,634,0,1,"Parr, Mr. William Henry Marsh",male,NaN,0,0,112052,0.00,NaN,S
815,816,0,1,"Fry, Mr. Richard",male,NaN,0,0,112058,0.00,B102,S
766,767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.60,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...,...
563,564,0,3,"Simmons, Mr. John",male,NaN,0,0,SOTON/OQ 392082,8.05,NaN,S
87,88,0,3,"Slocovski, Mr. Selman Francis",male,NaN,0,0,SOTON/OQ 392086,8.05,NaN,S
783,784,0,3,"Johnston, Mr. Andrew G",male,NaN,1,2,W./C. 6607,23.45,NaN,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S


В имени может присутствовать обращение. Изучим их.

In [63]:
train_df['Honorific'] = train_df['Name'].str.extract(r', ([a-zA-Z]+\.*) ')

honorifics_count = train_df['Honorific'].value_counts().reset_index()
honorifics_count.columns = ['Honorific', 'Count']

print(honorifics_count)

    Honorific  Count
0         Mr.    517
1       Miss.    182
2        Mrs.    125
3     Master.     40
4         Dr.      7
5        Rev.      6
6       Mlle.      2
7      Major.      2
8        Col.      2
9         the      1
10      Capt.      1
11        Ms.      1
12       Sir.      1
13      Lady.      1
14       Mme.      1
15       Don.      1
16  Jonkheer.      1


Master. - для детей мужского пола до 13 лет. Для усреднения приведём всех с этим обращением и пустым возрастом к 7 годам.\
Miss. - для незамужних девушек. Приведём к 9.\
\
Средняя продолжитльность жизни в Великобритании в 1912:\
Мужчины: ~48 лет\
Женщины: ~52 лет\
\
Mrs. - замужняя женщина. Приведём к 36.\
Mr. - женатый мужчина. Приведём к 34.\
\
Остальных - приведём к 30.

In [64]:
age_fix = {
    'Master.': 7,
    'Miss.': 9,
    'Mrs.': 36,
    'Mr.': 34
}

train_df['Age'] = train_df['Honorific'].map(age_fix).fillna(train_df['Age'])

age_is_null = train_df['Age'].isnull()
train_df.loc[age_is_null, 'Age'] = 30

Удалим колонку 'Honorific'.

In [65]:
train_df.drop('Honorific', axis=1, inplace=True)

Изучим колонку 'Cabin'.

In [66]:
train_df[train_df['Cabin'].isna()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,34.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,9.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,34.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,34.0,0,0,330877,8.4583,NaN,Q
7,8,0,3,"Palsson, Master. Gosta Leonard",male,7.0,3,1,349909,21.0750,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
884,885,0,3,"Sutehall, Mr. Henry Jr",male,34.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,36.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,9.0,1,2,W./C. 6607,23.4500,NaN,S


Непонятно что с ней делать, поэтому заполним пустые ячейки таким значением, чтобы модель их игнорировала.

In [67]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

Ещё раз проверяем пустые. Видим, что их нет.

In [68]:
null_value_stats = train_df.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

Разделяем датасет для обучения на две части - данные (X) и целевая метрика (y). Модель будет учиться определять 'y'.

In [69]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

Изучаем типы данных в колонках. Нужно определить, какие из них категориальные.

In [70]:
print(X.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


Категориальные данные записываем в массив 'categorical_features_indexes'.

Важно! Всё, что не числовые колонки, должно быть в этом списке.

In [71]:
categoricals_list = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Name', 'Ticket', 'Embarked', 'Cabin', 'PassengerId']
# categoricals_list = ['Sex', 'Name', 'Ticket', 'Embarked', 'Cabin', 'PassengerId'] # Другой вариант

categorical_features_indexes = np.array([X.columns.get_loc(col) for col in categoricals_list])

Есть смысл игнорировать часть колонок. Ненужные поместим в отдельный список, чтобы передать в модель при обучении через параметр 'ignored_features'.

На последующих шагах, в результате нескольких обучений, получаем результат - 'Name', 'Ticket', 'Embarked', 'Cabin' - не оказывают значимого влияния:\
Sex: 59.35579560\
Fare: 12.48467673\
Pclass: 12.45593200\
SibSp: 5.78339522\
Parch: 5.41685146\
Age: 4.50334900\
Ticket: 0.00000000\
Name: 0.00000000\
Embarked: 0.00000000\
Cabin: 0.00000000

Удалим колонку 'Name' - зависимость результата от имени не кажется возможной.\
Также поступим с 'Ticket', 'Embarked', 'Cabin'.

In [72]:
# train_df.drop('Name', axis=1, inplace=True)
# train_df.drop('Ticket', axis=1, inplace=True)
# train_df.drop('Embarked', axis=1, inplace=True)
# train_df.drop('Cabin', axis=1, inplace=True)

# test_df.drop('Name', axis=1, inplace=True)
# test_df.drop('Ticket', axis=1, inplace=True)
# test_df.drop('Embarked', axis=1, inplace=True)
# test_df.drop('Cabin', axis=1, inplace=True)

ignored_column_names = ['Name', 'Ticket', 'Embarked', 'Cabin', 'PassengerId']
ignored_features_indexes = np.array([X.columns.get_loc(col) for col in ignored_column_names])

# Данные для валидации
Разделим обучающие данные на две части - для обучения и валидации.

In [73]:
X_train, X_validation, y_train, y_validation = train_test_split(
    X
    , y
    , train_size=0.75
    , random_state=42
    , stratify=y # stratify=y означает, что делаем равномерное распределение величины 'y' - целевой метрики
    , shuffle=True
    )

X_test = test_df

# Обучаем модель
Задаём параметры для обучения.

In [74]:
params = {
    'random_seed': 42 # Если использовать одинаковый, то результат всегда будет повторяемый
    , 'logging_level': 'Silent' # Без вывода логов
    , 'use_best_model': True # В итоговой модели последний шаг тот, у которого лучший результат, а не тот, что в 'iterations'
    # , 'task_type': 'GPU' # Для расчёта на GPU
    , 'loss_function': metrics.Logloss()
    , 'ignored_features': ignored_features_indexes

    # Кол-во шагов (итераций) и скорость обучения связаны. В идеале 
    # , 'iterations': 1000 # По умолчанию 1000
    # , 'learning_rate': 0.01 # Можно задать скорость обучения

    , 'iterations': 1000
    , 'learning_rate': 0.028682999312877655

    # По этому параметру определяем лучшую модель
    # , 'eval_metric': metrics.Accuracy() # Другой синтаксис
    , 'eval_metric': 'Accuracy'
    # , 'eval_metric': 'AUC'
    # , 'eval_metric': 'F1'

    # Это метрика, которую рисуем для информации на графике. НЕ влияет на выбор лучшей модели
    , 'custom_loss':[metrics.AUC(), metrics.F1()] # Другой синтаксис
    # , 'custom_metric':[metrics.Accuracy()] # Другой синтаксис

    # Ранняя остановка - отсутствие изменений или контроль переобучения
    # Либо 'early_stopping_rounds' либо 'od_type'+'od_wait'
    # , 'early_stopping_rounds': 100 # Кол-во шагов
    , 'od_type': 'Iter' # Тип - считаем шаги
    , 'od_wait': 150 # Кол-во шагов
}

Pool - структура данных catboost. Формируем 2 штуки - для обучения и валидации.\
\
Дополнительно передаём в него категориальные данные - отдельно индексы, отдельно названия.

In [75]:
train_pool    = Pool(X_train,      y_train,      cat_features=categorical_features_indexes, feature_names=list(X.columns))
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indexes, feature_names=list(X.columns))

Выбираем тип обучения 'CatBoostClassifier' - это модель для классификации. Хорошо предсказывает бинарную метрику.\
Тут наша целевая метрика: 1 или 0, выжил или не выжил.\
\
В функцию 'CatBoostClassifier' - передаём параметры.\
Метод 'fit' - это запуск обучения. В него передаём Pool.\
\
Параметр 'plot' в методе 'fit' - рисует график.

In [76]:
model = CatBoostClassifier(**params)
model.fit(train_pool, eval_set=validate_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

**Модель обучена!**

# Оценка результата обучения
Изучим влияние колонок.

In [77]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {:.8f}'.format(name, score))

Sex: 58.64871000
Pclass: 22.12684557
Fare: 10.26032564
Age: 7.89052599
SibSp: 1.07359281
Ticket: 0.00000000
PassengerId: 0.00000000
Parch: 0.00000000
Name: 0.00000000
Embarked: 0.00000000
Cabin: 0.00000000


In [78]:
print('tree_count: ' + str(model.tree_count_))
print('learning_rate: ' + str(model.learning_rate_))
print('best_iteration: ' + str(model.get_best_iteration()))
print('best_score: ' + str(model.get_best_score()))

print('Model validation accuracy: {:.4}'.format(accuracy_score(y_validation, model.predict(X_validation))))

# print('Model auc: {:.4}'.format(auc(y_validation, model.predict(X_validation))))
print('Model f1_score: {:.4}'.format(f1_score(y_validation, model.predict(X_validation))))
print('Model log_loss: {:.4}'.format(log_loss(y_validation, model.predict(X_validation))))
print('Model recall_score: {:.4}'.format(recall_score(y_validation, model.predict(X_validation))))
print('Model roc_auc_score: {:.4}'.format(roc_auc_score(y_validation, model.predict(X_validation))))
print('Model precision_score: {:.4}'.format(precision_score(y_validation, model.predict(X_validation))))
# print('Model roc_curve: {:.4}'.format(roc_curve(y_validation, model.predict(X_validation))))

tree_count: 11
learning_rate: 0.028682999312877655
best_iteration: 10
best_score: {'learn': {'Accuracy': 0.8652694610778443, 'F1': 0.8117154811715481, 'Logloss': 0.342679299864937}, 'validation': {'Accuracy': 0.820627802690583, 'F1': 0.7468354430379746, 'Logloss': 0.4359726859111996, 'AUC': 0.8570701069427941}}
Model validation accuracy: 0.8206
Model f1_score: 0.7468
Model log_loss: 6.465
Model recall_score: 0.686
Model roc_auc_score: 0.7956
Model precision_score: 0.8194


Как модель показывает себя на валидационном датасете.

In [79]:
# model = CatBoostClassifier(iterations=50, random_seed=42, logging_level='Silent').fit(train_pool)
# eval_metrics = model.eval_metrics(data=validate_pool, metrics=[metrics.AUC()], plot=True)
eval_metrics = model.eval_metrics(data=validate_pool, metrics=['Accuracy', 'AUC', 'F1', 'Logloss'], plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [80]:
# model.plot_tree(tree_idx=18, pool=pool) # Требует дополнительных библиотек

In [81]:
# Можно сохранить в разных форматах

# model.save_model('catboost_model.dump')
# model.save_model('catboost_model.cbm', format='cbm')
# model.save_model('catboost_model.json', format='json')

# Проверим
Обучим вторую модель с измененными параметрами и сравним их.

In [82]:
# params.update({
#     'random_seed': 69
# })

# model2 = CatBoostClassifier(**params)

# model2.fit(train_pool, eval_set=validate_pool, plot=True)

In [83]:
# model.compare(model=model2, data=validate_pool, metrics=['Accuracy', 'AUC', 'Logloss'])

# Кросс-валидация
Проведём кросс-валидацию для дополнительной проверки

In [84]:
# cv_params = model.get_params()

# cv_pool = Pool(X, y, cat_features=categorical_features_indexes, feature_names=list(X.columns))

# cv_data = cv(
#     cv_pool
#     , cv_params
#     , plot=True
#     , fold_count=10 # По умолчанию 3
# )

Сделаем несолько фолдов и посмотрим какие результаты покажут модели.

In [85]:
# print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
#     np.max(cv_data['test-Accuracy-mean']),
#     cv_data['test-Accuracy-std'][np.argmax(cv_data['test-Accuracy-mean'])],
#     np.argmax(cv_data['test-Accuracy-mean'])
# ))

In [86]:
# print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Сравним результат кросс-валидации с первоначальным результатом.

### 2.3 Model Applying
All you have to do to get predictions is

In [87]:
predictions = model.predict(X_test)
predictions_probs = model.predict_proba(X_test)
print(predictions[:10])
print(predictions_probs[:10])

[0 1 0 0 1 1 1 0 1 0]
[[0.63545405 0.36454595]
 [0.48559123 0.51440877]
 [0.62099267 0.37900733]
 [0.5873478  0.4126522 ]
 [0.49682747 0.50317253]
 [0.49376389 0.50623611]
 [0.46743337 0.53256663]
 [0.57647345 0.42352655]
 [0.45901277 0.54098723]
 [0.56608667 0.43391333]]


### 3.10 Learning Processes Comparison
You can also compare different models learning process on a single plot.

In [88]:
# model1 = CatBoostClassifier(iterations=100, depth=1, train_dir='model_depth_1/', logging_level='Silent')
# model1.fit(train_pool, eval_set=validate_pool)
# model2 = CatBoostClassifier(iterations=100, depth=5, train_dir='model_depth_5/', logging_level='Silent')
# model2.fit(train_pool, eval_set=validate_pool);

In [89]:

# widget = MetricVisualizer(['model_depth_1', 'model_depth_5'])
# widget.start()

# $$4.\ Parameters\ Tuning$$
While you could always select optimal number of iterations (boosting steps) by cross-validation and learning curve plots, it is also important to play with some of model parameters, and we would like to pay some special attention to `l2_leaf_reg` and `learning_rate`.

In this section, we'll select these parameters using the **`hyperopt`** package.

In [90]:


# def hyperopt_objective(params):
#     model = CatBoostClassifier(
#         l2_leaf_reg=int(params['l2_leaf_reg']),
#         learning_rate=params['learning_rate'],
#         iterations=500,
#         eval_metric=metrics.Accuracy(),
#         random_seed=42,
#         verbose=False,
#         loss_function=metrics.Logloss(),
#     )
    
#     cv_data = cv(
#         Pool(X, y, cat_features=categorical_features_indexes),
#         model.get_params(),
#         logging_level='Silent',
#     )
#     best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
#     return 1 - best_accuracy # as hyperopt minimises

In [91]:


# params_space = {
#     'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
#     'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
# }

# trials = hyperopt.Trials()

# best = hyperopt.fmin(
#     hyperopt_objective,
#     space=params_space,
#     algo=hyperopt.tpe.suggest,
#     max_evals=50,
#     trials=trials,
#     rstate=default_rng(123)
# )

# print(best)

Now let's get all cv data with best parameters:

In [92]:
# model = CatBoostClassifier(
#     l2_leaf_reg=int(best['l2_leaf_reg']),
#     learning_rate=best['learning_rate'],
#     iterations=500,
#     eval_metric=metrics.Accuracy(),
#     random_seed=42,
#     verbose=False,
#     loss_function=metrics.Logloss(),
# )
# cv_data = cv(Pool(X, y, cat_features=categorical_features_indexes), model.get_params())

In [93]:
# print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

Recall that with default parameters out cv score was 0.8283, and thereby we have (probably not statistically significant) some improvement.

### Make submission
Now we would re-train our tuned model on all train data that we have

In [94]:
final_pool = Pool(X, y, cat_features=categorical_features_indexes, feature_names=list(X.columns))

model2 = CatBoostClassifier(**params)
model2.fit(
    final_pool
    , eval_set=validate_pool
    # , plot=True
    )

In [95]:
print('tree_count: ' + str(model2.tree_count_))
print('Model validation accuracy: {:.4}'.format(accuracy_score(y_validation, model2.predict(X_validation))))

tree_count: 78
Model validation accuracy: 0.843


In [96]:
# model3 = CatBoostClassifier(**params)
# model3.fit(
#     train_pool
#     , eval_set=validate_pool
#     # , plot=True
#     )

In [97]:
# print('tree_count: ' + str(model3.tree_count_))
# print('Model validation accuracy: {:.4}'.format(accuracy_score(y_validation, model3.predict(X_validation))))

# Оценка результата обучения Финальной модели
Изучим влияние колонок.

In [98]:
# feature_importances = model.get_feature_importance(train_pool)
# feature_names = X_train.columns
# for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
#     print('{}: {:.8f}'.format(name, score))

In [99]:
# print('tree_count: ' + str(model.tree_count_))
# print('learning_rate: ' + str(model.learning_rate_))
# print('best_iteration: ' + str(model.get_best_iteration()))
# print('best_score: ' + str(model.get_best_score()))

# print('Model validation accuracy: {:.4}'.format(accuracy_score(y_validation, model.predict(X_validation))))

# # print('Model auc: {:.4}'.format(auc(y_validation, model.predict(X_validation))))
# print('Model f1_score: {:.4}'.format(f1_score(y_validation, model.predict(X_validation))))
# print('Model log_loss: {:.4}'.format(log_loss(y_validation, model.predict(X_validation))))
# print('Model recall_score: {:.4}'.format(recall_score(y_validation, model.predict(X_validation))))
# print('Model roc_auc_score: {:.4}'.format(roc_auc_score(y_validation, model.predict(X_validation))))
# print('Model precision_score: {:.4}'.format(precision_score(y_validation, model.predict(X_validation))))
# # print('Model roc_curve: {:.4}'.format(roc_curve(y_validation, model.predict(X_validation))))

And finally let's prepare the submission file:

In [100]:
submisstion = pd.DataFrame()
submisstion['PassengerId'] = X_test['PassengerId']
submisstion['Survived'] = model2.predict(X_test)

In [101]:
submisstion.to_csv('submission.csv', index=False)

In [ ]:
# result_check = X_test
# print(result_check.head(5))

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  Survived  
0  34.5      0      0   330911   7.8292  -999        Q         0  
1  47.0      1      0   363272   7.0000  -999        S         1  
2  62.0      0      0   240276   9.6875  -999        Q         0  
3  27.0      0      0   315154   8.6625  -999        S         0  
4  22.0      1      1  3101298  12.2875  -999        S         1  


In [ ]:
# result_check['Survived2'] = model2.predict(X_test)
# result_check['Survived3'] = model3.predict(X_test)

In [ ]:
# result_check.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Survived2,Survived3
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,-999,Q,0,0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,-999,S,1,1,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,-999,Q,0,0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,-999,S,0,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,-999,S,1,1,1


In [ ]:
# result_check.loc[result_check['Survived2'] != result_check['Survived3']]
# result_check.loc[result_check['Survived2'] != result_check['Survived']]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Survived2,Survived3


Finally you can make submission at [Titanic Kaggle competition](https://www.kaggle.com/c/titanic).

That's it! Now you can play around with CatBoost and win some competitions! :)